In [ ]:
# Install core packages for fine-tuning
!pip install -q -U bitsandbytes==0.42.0
!pip install -q -U peft==0.15.1
!pip install -q -U trl==0.7.10
!pip install -q -U accelerate==0.27.1
!pip install -q -U datasets==2.17.0
!pip install -q -U transformers==4.41.1

# Fix gcsfs <-> fsspec conflict in Colab
!pip install -q fsspec==2025.3.2 gcsfs==2025.3.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.0/411.0 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 104.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 93.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 108.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from datasets import load_dataset

# Load a small instruction-tuning dataset (you can replace this later)
dataset = load_dataset("mlabonne/guanaco-llama2-1k", split="train")
dataset = dataset.train_test_split(test_size=0.1)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
from huggingface_hub import login

# Login using your Hugging Face token
login(token="put your HuggingFace code")

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "google/gemma-2b"
hf_token = "put your hugging face code"  # Your Hugging Face token

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id, token=hf_token)
tokenizer.pad_token = tokenizer.eos_token

# Load model with float16 (no quantization)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",             # automatically assign to GPU
    torch_dtype=torch.float16,     # efficient and fast on GPU
    token=hf_token
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [ ]:
!pip uninstall -y peft bitsandbytes
!pip install -q peft==0.10.0 accelerate==0.27.2

Found existing installation: peft 0.10.0
Uninstalling peft-0.10.0:
  Successfully uninstalled peft-0.10.0


In [ ]:
import sys
import types

# Create full fake bitsandbytes structure
bnb = types.ModuleType("bitsandbytes")
bnb.nn = types.ModuleType("bitsandbytes.nn")
bnb.nn.modules = types.ModuleType("bitsandbytes.nn.modules")
bnb.nn.Linear8bitLt = object
bnb.functional = types.ModuleType("bitsandbytes.functional")
bnb.functional.dequantize_4bit = lambda x, **kwargs: x
bnb.functional.quantize_4bit = lambda x, **kwargs: x

sys.modules["bitsandbytes"] = bnb
sys.modules["bitsandbytes.nn"] = bnb.nn
sys.modules["bitsandbytes.nn.modules"] = bnb.nn.modules
sys.modules["bitsandbytes.nn.modules.Linear8bitLt"] = bnb.nn.Linear8bitLt
sys.modules["bitsandbytes.functional"] = bnb.functional

In [ ]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 921,600 || all params: 2,507,094,016 || trainable%: 0.036759690467068624


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",                     # Where to save outputs
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    logging_steps=10,
    num_train_epochs=1,                         # For demo; increase as needed
    learning_rate=2e-4,
    save_steps=50,
    save_total_limit=1,
    logging_dir="./logs",
    report_to="none",
    fp16=False,                                 # Use False on Colab CPU
)

In [ ]:
!pip uninstall -y bitsandbytes
!pip install -q --upgrade pip
!pip install -q \
  peft==0.8.2 \
  trl==0.7.10 \
  transformers==4.41.1 \
  accelerate==0.27.2 \
  datasets==2.18.0

Found existing installation: bitsandbytes 0.40.2
Uninstalling bitsandbytes-0.40.2:
  Successfully uninstalled bitsandbytes-0.40.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 23.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 2.7.19 requires torch<2.7,>=1.10, but you have torch 2.7.1 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 122.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 127.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10/10 [peft]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dask-cudf-cu12 25.2.2 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.0 which is incompatible.
gcsfs 2025.3.2 requires fsspec==2025.

In [ ]:
!pip uninstall -y transformers accelerate
!pip install -q transformers==4.41.1 accelerate==0.27.2

Found existing installation: transformers 4.41.1
Uninstalling transformers-4.41.1:
  Successfully uninstalled transformers-4.41.1
Found existing installation: accelerate 0.27.2
Uninstalling accelerate-0.27.2:
  Successfully uninstalled accelerate-0.27.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [accelerate]


In [1]:
!pip install transformers==4.31.0
!pip install trl==0.4.7

  Using cached trl-0.4.7-py3-none-any.whl.metadata (10 kB)
Using cached trl-0.4.7-py3-none-any.whl (77 kB)
  Attempting uninstall: trl
    Found existing installation: trl 0.7.10
    Uninstalling trl-0.7.10:
      Successfully uninstalled trl-0.7.10


In [1]:
!pip install transformers==4.31.0
!pip install trl==0.4.7
!pip install peft==0.4.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 7.7 MB/s eta 0:00:00
  Attempting uninstall: peft
    Found existing installation: peft 0.8.2
    Uninstalling peft-0.8.2:
      Successfully uninstalled peft-0.8.2


Restart the session here

In [6]:
from datasets import load_dataset
from transformers import AutoTokenizer, TrainingArguments, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model, TaskType
from trl import SFTTrainer
import torch

# Load small dataset for testing
dataset = load_dataset("Abirate/english_quotes", split="train")

# Model and tokenizer
model_name = "tiiuae/falcon-rw-1b"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # set padding

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto" if torch.cuda.is_available() else None
)

# Correct LoRA config for Falcon
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["query_key_value"],  # Fixed for Falcon
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
)

# Apply LoRA
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    logging_steps=10,
    num_train_epochs=1,  # You can increase this for better results
    save_strategy="epoch",
    save_total_limit=1,
    fp16=torch.cuda.is_available(),
    report_to="none"
)

# SFTTrainer setup
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=lora_config,
    dataset_text_field="quote",
    tokenizer=tokenizer,
    args=training_args,
    max_seq_length=512,
)

# Start training
trainer.train()

trainable params: 1,572,864 || all params: 1,313,198,080 || trainable%: 0.11977355312612092


Map:   0%|          | 0/2508 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/accelerate/accelerator.py:450: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
10,2.155500
20,2.052500
30,1.993500
40,2.099500
50,2.059200
60,1.838000
70,2.175400
80,1.896200
90,1.873400
100,1.778500


TrainOutput(global_step=313, training_loss=1.9967893685776585, metrics={'train_runtime': 127.431, 'train_samples_per_second': 19.681, 'train_steps_per_second': 2.456, 'total_flos': 1103171771351040.0, 'train_loss': 1.9967893685776585, 'epoch': 1.0})

In [7]:
# Save the LoRA adapter and tokenizer
model.save_pretrained("./lora-falcon-rw-1b")
tokenizer.save_pretrained("./lora-falcon-rw-1b")

('./lora-falcon-rw-1b/tokenizer_config.json',
 './lora-falcon-rw-1b/special_tokens_map.json',
 './lora-falcon-rw-1b/vocab.json',
 './lora-falcon-rw-1b/merges.txt',
 './lora-falcon-rw-1b/added_tokens.json',
 './lora-falcon-rw-1b/tokenizer.json')

In [11]:
from transformers import TextStreamer

input_text = "what is the purpose of a car"
inputs = tokenizer(input_text, return_tensors="pt").to(model.device)

# Generate
outputs = model.generate(
    **inputs,
    max_new_tokens=50,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    temperature=0.9,
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


what is the purpose of a car stereo?
A car stereo is an important part of your vehicle. It allows you to enjoy your music and your favourite movies even when you are in your car. You can listen to your favourite songs and movies even while driving. The best part about
